#### Initialize ####

In [23]:
function tf {aws-vault exec terraform_user -- terraform $args}

function prompt_to_confirm { 
    $user_response = Read-host "Continue? y/n or 1/0"
    if ($user_response -ne "y" -and $user_response -ne 1) {break}
}

In [24]:
$aws_region = "us-east-2"
$env:AWS_REGION=$aws_region #for aws_vault
$env:TF_VAR_aws_region=$aws_region #for terraform

In [34]:
terraform init #-upgrade

Initializing the backend...
Initializing modules...
- one_webserver in .terraform\modules\one_webserver\configurations_modules\one_webserver
Initializing provider plugins...
- Reusing previous version of hashicorp/local from the dependency lock file
- Reusing previous version of hashicorp/tls from the dependency lock file
- Reusing previous version of hashicorp/aws from the dependency lock file
- Using previously-installed hashicorp/local v2.5.2
- Using previously-installed hashicorp/tls v4.0.6
- Using previously-installed hashicorp/aws v5.70.0

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


#### Workflow ####

In [26]:
terraform fmt

In [27]:
terraform validate

Success! The configuration is valid.



In [28]:
$keyname = "ssh_key"
$env:TF_VAR_ssh_keyname=$keyname
#ssh-keygen -t rsa -b 4096 -f $keyname -N "" -C $keyname.pem

In [38]:
###      ###
### PLAN ###
###      ###

tf plan -out=tfplan
terraform show -json tfplan > plan.json
Get-Date

data.aws_ami.ami: Reading...
module.one_webserver.data.aws_subnet.public_subnet: Reading...
module.one_webserver.data.aws_vpc.existing_vpc: Reading...
data.aws_ami.ami: Read complete after 1s [id=ami-09da212cf18033880]
module.one_webserver.data.aws_subnet.public_subnet: Read complete after 1s [id=subnet-0c13b6812ab6e3f89]
module.one_webserver.data.aws_vpc.existing_vpc: Read complete after 2s [id=vpc-0a4860611e96d1d85]

Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # local_file.private_key will be created
  + resource "local_file" "private_key" {
      + content              = (sensitive value)
      + content_base64sha256 = (known after apply)
      + content_base64sha512 = (known after apply)
      + content_md5          = (known after apply)
      + content_sha1         = (known after apply)
      + content_sha256       = (known aft

In [39]:
#Output planned resource addresses and their names in the cloud

$plan = Get-Content -Raw -Path "plan.json" | ConvertFrom-Json

$resourcesToDeploy = $plan.resource_changes | Where-Object { 
    $_.change.actions -contains "create" -or $_.change.actions -contains "update" 
} | ForEach-Object { 
    "Type: " + $_.type
    "TF name: " + $_.name
    "Cloud name: " + $_.change.after.name
    "Name tag: " + $_.change.after.tags_all.name
    ""
}

$resourcesToDeploy
Get-Date

Type: local_file
TF name: private_key
Cloud name: 
Name tag: 

Type: aws_instance
TF name: example
Cloud name: 
Name tag: one_webserver-prod

Type: aws_key_pair
TF name: generated_key
Cloud name: 
Name tag: 

Type: aws_security_group
TF name: instance
Cloud name: example_security_group
Name tag: 

Type: aws_security_group_rule
TF name: allow_8080
Cloud name: 
Name tag: 

Type: aws_security_group_rule
TF name: allow_all_egress
Cloud name: 
Name tag: 

Type: aws_security_group_rule
TF name: allow_ssh
Cloud name: 
Name tag: 

Type: tls_private_key
TF name: ssh_key
Cloud name: 
Name tag: 


Tuesday, October 15, 2024 3:21:45 PM



In [40]:
###       ###
### APPLY ###
###       ###

#try {prompt_to_confirm} catch {break}
tf apply -auto-approve
Get-Date

module.one_webserver.data.aws_vpc.existing_vpc: Reading...
module.one_webserver.data.aws_subnet.public_subnet: Reading...
data.aws_ami.ami: Reading...
module.one_webserver.data.aws_subnet.public_subnet: Read complete after 1s [id=subnet-0c13b6812ab6e3f89]
data.aws_ami.ami: Read complete after 1s [id=ami-09da212cf18033880]
module.one_webserver.data.aws_vpc.existing_vpc: Read complete after 2s [id=vpc-0a4860611e96d1d85]

Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # local_file.private_key will be created
  + resource "local_file" "private_key" {
      + content              = (sensitive value)
      + content_base64sha256 = (known after apply)
      + content_base64sha512 = (known after apply)
      + content_md5          = (known after apply)
      + content_sha1         = (known after apply)
      + content_sha256       = (known aft

In [41]:
$public_ip = terraform output -raw public_ip
$http_port = terraform output -raw http_port

In [42]:
(Invoke-WebRequest -Uri ("http://" + $public_ip + ":$http_port")).Content

https://github.com/aksenov-fx/



In [ ]:
###         ###
### DESTROY ###
###         ###

try {prompt_to_confirm} catch {break}
tf destroy -auto-approve
Get-Date

#### Info ####

In [43]:
terraform version

Terraform v1.9.5
on windows_amd64
+ provider registry.terraform.io/hashicorp/aws v5.70.0
+ provider registry.terraform.io/hashicorp/local v2.5.2
+ provider registry.terraform.io/hashicorp/tls v4.0.6

Your version of Terraform is out of date! The latest version
is 1.9.7. You can update by downloading from https://www.terraform.io/downloads.html


In [44]:
#Root module configuration
cat .\main.tf

terraform {
  required_version = ">= 1.9.5, < 2.0"

  required_providers {
    aws = {
      source = "hashicorp/aws"
    }
    local = {
      source = "hashicorp/local"
    }
    tls = {
      source = "hashicorp/tls"
    }
  }
}

provider "aws" {
  region = var.aws_region
}

# --- --- --- --- --- --- --- --- --- --- #

# Get ami_id
data "aws_ami" "ami" {
  most_recent = true
  owners      = ["137112412989"] #Amazon
  filter {
    name   = "name"
    values = ["al2023-ami-2023.5*"] #Amazon_Linux_2023
  }
}

# --- --- --- --- --- --- --- --- --- --- #

resource "local_file" "private_key" {
  content  = module.one_webserver.private_key_pem
  filename = "./${var.ssh_keyname}.pem"
}

# --- --- --- --- --- --- --- --- --- --- #

module "one_webserver" {
  source = "git::https://github.com/aksenov-fx/terraform-modules.git//configurations_modules/one_webserver"
  #source = "../../configurations_modules/one_webserver"

  ami_id        = data.aws_ami.ami.id
  instance_type = "t2.micro"

  vpc

In [45]:
($resources = terraform state list) | Out-Host

data.aws_ami.ami
local_file.private_key
module.one_webserver.data.aws_subnet.public_subnet
module.one_webserver.data.aws_vpc.existing_vpc
module.one_webserver.aws_instance.example
module.one_webserver.aws_key_pair.generated_key
module.one_webserver.aws_security_group.instance
module.one_webserver.aws_security_group_rule.allow_8080
module.one_webserver.aws_security_group_rule.allow_all_egress
module.one_webserver.aws_security_group_rule.allow_ssh
module.one_webserver.tls_private_key.ssh_key


In [ ]:
foreach ($resource in $resources) { terraform state show $resource }

In [46]:
terraform output

http_port = 8080
public_ip = "3.141.2.241"


#### Remote access ####

In [47]:
Test-NetConnection -ComputerName $public_ip -Port 22


ComputerName     : 3.141.2.241
RemoteAddress    : 3.141.2.241
RemotePort       : 22
InterfaceAlias   : Ethernet
SourceAddress    : 192.168.0.23
TcpTestSucceeded : True



Error: Command failed: SubmitCode: Test-NetConnection -ComputerName $public_ip -Port  ...

In [48]:
puttygen "$keyname.pem" -O private -o "$keyname.ppk"

In [50]:
#Start SSH session

#Start ssh -A "ec2-user@$public_ip -i $keyname.pem"
Start "plink.exe" -A "-ssh -i $keyname.ppk ec2-user@$public_ip"

In [52]:
#Execute single SSH command

#ssh ec2-user@$public_ip -i "$keyname.pem" 'cat /etc/xattr.conf'
plink -ssh -i "$keyname.ppk" "ec2-user@$public_ip" "cat /etc/os-release"

NAME="Amazon Linux"
VERSION="2023"
ID="amzn"
ID_LIKE="fedora"
VERSION_ID="2023"
PLATFORM_ID="platform:al2023"
PRETTY_NAME="Amazon Linux 2023.5.20241001"
ANSI_COLOR="0;33"
CPE_NAME="cpe:2.3:o:amazon:amazon_linux:2023"
HOME_URL="https://aws.amazon.com/linux/amazon-linux-2023/"
DOCUMENTATION_URL="https://docs.aws.amazon.com/linux/"
SUPPORT_URL="https://aws.amazon.com/premiumsupport/"
BUG_REPORT_URL="https://github.com/amazonlinux/amazon-linux-2023"
VENDOR_NAME="AWS"
VENDOR_URL="https://aws.amazon.com/"
SUPPORT_END="2028-03-15"


#### Troubleshooting ####

In [ ]:
#$env:TF_LOG = "TRACE"
#$env:TF_LOG_PATH = "log.txt"
#$env:TF_LOG = ""

In [ ]:
tf apply -refresh-only -auto-approve